In [2]:
! pip install openpyxl gensim fasttext lightgbm multipledispatch razdel

In [3]:
import pandas as pd
import numpy as np
import fasttext
import razdel

import fasttext
from gensim.models import FastText

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, average_precision_score
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
import joblib

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [4]:
columns = ['Общее наименование продукции', 'Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)', 'Подкатегория продукции']
big = pd.read_excel('/kaggle/input/hackaton/big.xlsx', sheet_name='все ДС с кодами')[columns]
small = pd.read_excel('/kaggle/input/hackaton/small.xlsx', sheet_name='Данные для сопоставления')[columns]
big = big.rename(columns={"Общее наименование продукции": "product_name", 
                   "Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)": "level",
                   "Подкатегория продукции": "category"})

small = small.rename(columns={"Общее наименование продукции": "product_name", 
                   "Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)": "level",
                   "Подкатегория продукции": "category"})

all_data = pd.concat([big, small])

In [5]:
all_data['level_2'] = all_data['level'].str.split(';')
all_data['category'] = all_data['category'].str.split(';')

In [6]:
all_data = all_data.set_index(['product_name', 'level']).explode(['level_2', 'category']).reset_index()
all_data['level_1'] = all_data['level_2'].apply(lambda x: x.split('.')[0]).str.strip()
all_data['level_2'] = all_data['level_2'].str.strip()
all_data['category'] = all_data['category'].str.strip()

In [7]:
all_data.shape

In [8]:
dictionary = all_data[['category', 'level_1', 'level_2']].drop_duplicates()

In [9]:
dictionary.shape

In [10]:
dictionary.to_csv('dictionary_level_2.csv', sep=';', index=None)

In [11]:
all_data.shape

## Предобработка

In [12]:
import re
russian_stopwords = open('/kaggle/input/hackaton/stopwords-ru.txt', 'r').read().split('\n')
okpd = pd.read_excel('/kaggle/input/hackaton/okpd.xlsx').drop_duplicates(subset=['okpd'])
okpd = okpd[okpd['okpd'].str.len()>10].reset_index(drop=True).reset_index()
dictionary = pd.read_csv('/kaggle/input/hackaton/levels.csv', sep=';', encoding='cp1251').dropna(subset=['level_2'])
dictionary['level'] = dictionary['level_1'].astype(str) + '.' + dictionary['level_2'].astype(int).astype(str)
dictionary = dictionary.drop_duplicates(subset='level')

def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word.lower() not in russian_stopwords]).split()

def delete_punctuation(s):
    return re.sub(r'[®?"\'-_/.:?!1234567890()%<>;,+#$&№\s+]', u' ', s)

def get_okpd(line) -> int :
    okpd_re = re.compile('окпд2\x20*(\d{2}\.\d{2}\.\d{2}\.\d{3})')
    res = re.findall(okpd_re, line.lower())
    return res[0] if len(res) > 0 else None 

In [13]:
all_data['okpd'] = all_data['product_name'].apply(get_okpd)
all_data = all_data.join(okpd.set_index('okpd'), on = 'okpd')
all_data['clean_product_name'] = all_data['product_name'].apply(lambda x: ' '.join(delete_stopwords(delete_punctuation((x)))))

In [14]:
all_data.columns

## Определение категории статистикой

In [15]:
all_categories = all_data.groupby(['level_2', 'category'])['level'].count().reset_index()
all_categories = all_categories.rename(columns={'level':'count'})

In [16]:
def get_category_sim(product_name:str, category: str, all_categories):
    all_categories['sim'] = get_similarity(product_name, all_categories)
    all_categories = all_categories.sort_values(by=['sim', 'count'], ascending=False)
    label, probability = all_categories[['level_2', 'sim']].values[0]
    return label, probability, category==label

def get_similarity(product_name:str, all_categories):
    probability = []
    for category in all_categories:
        probability.append(fuzz.token_sort_ratio(short_rp_name, category)/100)
    return probability#all_categories['category'].apply(lambda x: ).values.tolist()

In [17]:
# all_categories_list = all_categories[all_categories['count']<50]['category'].tolist()
# print(len(all_categories_list))
# all_data['features'] = all_data['clean_product_name'].progress_apply(lambda x: get_similarity(x, all_categories_list ))

## Генерация фичей

In [ ]:
df_categ = all_data[['category','level']].value_counts().reset_index().rename(columns={0:'count'})


def new_category_sim(product_name:str, category: int, df_categ):
    short_rp_name = ' '.join(product_name.split()[:10])
    threshold = 10
    if category in df_categ[df_categ['count']<threshold]['category'].to_list():
        
        df_categ['sim'] = df_categ['level'].apply(lambda x: fuzz.token_sort_ratio(x, short_rp_name))
        
        max_sim = max(df_categ['sim'])
        
        max_cat = max(df_categ[df_categ['sim']==max_sim]['count'])
        print(max_sim,max_cat)
        
        res = df_categ[(df_categ['sim']==max_sim) & (df_categ['count']==max_cat)]['category'].values[0]
        return res

In [12]:
from scipy.spatial.distance import cosine
from nltk.util import ngrams
from fuzzywuzzy import fuzz


def texts_distance(text1, text2, model):
    try:
        A = get_vec(text1, model)
        B = get_vec(text2, model)
        return cosine(A, B)
    except:
        return 1


def get_vec(text, model):
    if len(text) != 0:
        try:
            return model.get_sentence_vector(text)
        except:
            return np.array([[0]*dim_ft])
    else:
        return np.array([[0]*dim_ft])
    
    
def get_features(data, column1, column2):
    #считаем пересечение между сетами сущностей двух названий
    data['intersection'] = data.progress_apply(lambda x: len(set(x[column1+'_tokens']) & set(x[column2+'_tokens'])), axis = 1)
    #считаем объединение сетов сущностей двух названий
    data['union'] = data.progress_apply(lambda x: len(set(x[column1+'_tokens']) | set(x[column2+'_tokens'])), axis = 1)
    #считаем коэф. жаккара пересечение/объединение
    data['jaccard'] = data.progress_apply(lambda x: x['intersection'] / x['union'] if x['union'] != 0 else 0, axis = 1)
    
    #считаем пересечение между сетами сущностей двух названий
    data['intersection_ngrams'] = data.progress_apply(lambda x: len(set(x[column1+'_bigrams']) & set(x[column1+'_bigrams'])), axis = 1)
    #считаем объединение сетов сущностей двух названий
    data['union_ngrams'] = data.progress_apply(lambda x: len(set(x[column1+'_bigrams']) | set(x[column2+'_bigrams'])), axis = 1)
    #считаем коэф. жаккара пересечение/объединение
    data['jaccard_ngrams'] = data.progress_apply(lambda x: x['intersection_ngrams'] / x['union_ngrams'] if x['union_ngrams'] !=0 else 0, axis = 1)    
    
                                               
    #считаем пересечение первых двух слов
    data['intersection_10w'] = data.progress_apply(lambda x: len(set(x[column1+'_first_10w']) & set(x[column2+'_first_10w'])), axis = 1)
    #считаем пересечение первых двух слов
    data['intersection_5w'] = data.progress_apply(lambda x: len(set(x[column1+'_first_5w']) & set(x[column2+'_first_5w'])), axis = 1)
    #считаем пересечение первых двух слов
    data['intersection_2w'] = data.progress_apply(lambda x: len(set(x[column1+'_first_2w']) & set(x[column2+'_first_2w'])), axis = 1)
    #считаем пересечение первых слов
    data['intersection_1w'] = data.progress_apply(lambda x: len(set(x[column1+'_first_1w']) & set(x[column2+'_first_1w'])), axis = 1)
    #считаем метрику похожести
    data['fuzzy_similarity_partial_left'] = data.fillna('').progress_apply(lambda x: fuzz.partial_ratio(x[column1], x[column2]), axis = 1)
    data['fuzzy_similarity'] = data.fillna('').progress_apply(lambda x: fuzz.ratio(x[column1], x[column2]), axis = 1)
    data['fuzzy_similarity_tokens'] = data.fillna('').progress_apply(lambda x: fuzz.token_set_ratio(x[column1], x[column2]), axis = 1)

    return data
                                                                                         
def get_ngrams(text, n = 2):
    if text == '':
        return []
    else:
        return [''.join(_) for _ in ngrams(text, n = n)]
                                                                                         
def tokenize(corpus):
    return [[word.text for word in razdel.tokenize(text.lower())] 
                       for text in corpus]

def get_first(df, n = 1):
    return df.progress_apply(lambda x: x[:n])

In [ ]:
all_data['product_name_tokens'] = tokenize(tqdm(all_data.product_name))
all_data['category_1_tokens'] = tokenize(tqdm(all_data.category_1.fillna('')))

In [ ]:
all_data['product_name_bigrams'] = all_data['product_name'].progress_apply(lambda x: get_ngrams(x, 2))
all_data['category_1_bigrams'] = all_data['category_1'].fillna('').progress_apply(lambda x: get_ngrams(x, 2))

In [ ]:
all_data['product_name_first_10w'] = get_first(all_data['product_name_tokens'], 10)
all_data['category_1_first_10w'] = get_first(all_data['category_1_tokens'], 10)
all_data['product_name_first_5w'] = get_first(all_data['product_name_tokens'], 5)
all_data['category_1_first_5w'] = get_first(all_data['category_1_tokens'], 5)
all_data['product_name_first_2w'] = get_first(all_data['product_name_tokens'], 2)
all_data['category_1_first_2w'] = get_first(all_data['category_1_tokens'], 2)
all_data['product_name_first_1w'] = get_first(all_data['product_name_tokens'], 1)
all_data['category_1_first_1w'] = get_first(all_data['category_1_tokens'], 1)

In [ ]:
all_data = get_features(all_data, 'product_name', 'category_1')

In [ ]:
all_data[['product_name', 'level', 'okpd', 'level_1', 'category_1', 'level_2',
       'category_2', 'name', 'clean_product_name', 
       'intersection', 'union', 'jaccard', 'intersection_ngrams',
       'union_ngrams', 'jaccard_ngrams', 'intersection_10w', 'intersection_5w',
       'intersection_2w', 'intersection_1w', 'fuzzy_similarity_partial_left',
       'fuzzy_similarity', 'fuzzy_similarity_tokens']].to_csv('dataset_with_features.csv', sep=';')

## Тестирование модели

## FastText обучение

In [18]:
open('all_text.txt', 'w').write('\n'.join(all_data['clean_product_name'].str.lower().str.strip().drop_duplicates().tolist()))

In [ ]:
%%time
fb_model = fasttext.train_unsupervised('/kaggle/working/all_text.txt',
                                       dim=300, ws=5, minCount=5, neg=15, 
                                       minn=3, maxn=5, wordNgrams=5, lr=0.15, epoch=25, loss='hs',
                                       model='skipgram', verbose=3)

In [20]:
fb_model.save_model('fb_model_deduplicated.bin') 

## Предсказание модели  

In [21]:
from sklearn.model_selection import train_test_split
import numpy as np
from lightgbm import LGBMClassifier
import joblib

down sampling

In [22]:
cnts = all_data['level_2'].value_counts()
normal_categories = cnts[cnts>10].index.tolist()

all_data = all_data[all_data['level_2'].isin(normal_categories)]
all_data = all_data.dropna(subset=['level_2'])
# all_data_filtred = pd.concat([ all_data[all_data.level_1==9300].drop_duplicates().sample(n=200000, replace=True, random_state=42),
#                                all_data[all_data.level_1!=9300]])

In [23]:
train, test = train_test_split(all_data[['clean_product_name', 'level_2']], test_size=0.3, random_state=42)
train, val = train_test_split(train[['clean_product_name', 'level_2']], test_size=0.15, random_state=42)

In [24]:
ft_model = fasttext.load_model('/kaggle/working/fb_model_deduplicated.bin')

In [25]:
%%time
train_vectors = np.array([ft_model.get_sentence_vector(text) for text in train['clean_product_name'].str.replace('\n', ' ').str.lower()])
val_vectors = np.array([ft_model.get_sentence_vector(text) for text in val['clean_product_name'].str.replace('\n', ' ').str.lower()])
test_vectors = np.array([ft_model.get_sentence_vector(text) for text in test['clean_product_name'].str.replace('\n', ' ').str.lower()])

In [26]:
lgbm_params = {
    'n_estimators': 300,
    'max_depth': 10,
    'learning_rate': 0.05,
    'n_jobs': 7,
    'random_state':42,
    'first_metric_only':True,
    'is_unbalance':True
}

model = LGBMClassifier(**lgbm_params)
model.fit(train_vectors, train['level_2'], verbose=True, eval_set=(val_vectors, val['level_2']),  early_stopping_rounds = 15)

In [27]:
from sklearn import metrics

y_predicted = model.predict(test_vectors)
print(metrics.classification_report(test['level_2'], y_predicted))

In [ ]:
y_predicted = model.predict(test_vectors)
print(metrics.classification_report(test['level_1'], y_predicted))

In [ ]:
joblib.dump(model, 'model.pkl')

## Предсказание для сервиса

In [ ]:
import fasttext
from lightgbm import LGBMClassifier
import joblib
from multipledispatch import dispatch

class ProductNameClassifier:
    def __init__(self):
        self.ft_model = fasttext.load_model('/kaggle/input/01-fasttext-prototype/fb_model.bin')
        self.model = joblib.load('model.pkl')
    
    @dispatch(str)
    def predict(self, text:str) -> (int, float):
        """
        Функция для обработки текста 
        """
        text = text.replace('\n', ' ')
        text_vector = self.ft_model.get_sentence_vector(text).reshape(1, -1)
        label = int(self.model.predict(text_vector)[0])
        probability = self.model.predict_proba(text_vector)
        return label, probability.max()  
    
    @dispatch(str, int)
    def predict(self, text:str, user_label:int) -> (str, float, int):
        """
        Функция для обработки текста и метки 
        """
        text = text.replace('\n', ' ')
        text_vector = self.ft_model.get_sentence_vector(text).reshape(1, -1)
        label = self.model.predict(text_vector)[0]
        probability = self.model.predict_proba(text_vector)
        is_equal = label == user_label
        return label, probability.max(), is_equal

In [ ]:
pd.DataFrame.from_dict({'product_name': {0: 'Парацетамол таблетки 500 мг 10 шт., упаковки ячейковые контурные (2), пачки картонные, рег № ЛС-001364 от 06.08.2010, серия 190618, партия 59110 упаковок, годен до 01.07.2022, производства  ОАО "Фармстандарт-Лексредства", ИНН 4631002737, 305022, Курская область, Курск, ул. 2-я Агрегатная, 1А/18, Россия, код ОКПД2 21.20.10.232 ',
  1: 'Перезаряжаемая литий-ионная батарея торговой марки HUAWEI модель HB642735ECW',
  2: 'Перезаряжаемая литий-ионная батарея торговой марки vivo модель B-E8',
  3: 'Аппарат вакуумно-лазерной терапии стоматологический АВЛТ-"ДЕСНА" (по методу Кулаженко-Лепилина)',
  4: 'Блоки оконные и балконные дверные из алюминиевых профилей системы "INICIAL" серии "IW63" фирмы ООО "Урало-Сибирская профильная компания"'},
 'category_1': {0: 9300, 1: 3482, 2: 3482, 3: 9444, 4: 5270},
 'model_label': {0: 9300.0, 1: 3482.0, 2: 3482.0, 3: 9441.0, 4: 5772.0},
 'probability': {0: 0.6942050498562966,
  1: 0.9989457655246328,
  2: 0.9989539612756103,
  3: 0.703270209758987,
  4: 0.9999890387091874}})

In [ ]:
product_name_clf = ProductNameClassifier()

#если есть и метка от пользователя и текст
label, probability, is_equal = product_name_clf.predict(big['product_name'][0], 9300)
print(label, probability, is_equal )

#если только текст
label, probability = product_name_clf.predict(big['product_name'][0])
print(label, probability)

#Скоринг всего файла с наличием метки от пользователя
df[['model_label', 'probability', 'is_equal']] = None
df[['model_label', 'probability', 'is_equal']] = df[['product_name', 'category_1']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")

#Скоринг всего файла без метки от пользователя
df[['model_label', 'probability']] = None
df[['model_label', 'probability']] = df[['product_name']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")

In [ ]:
multipledispatch==0.6.0
lightgbm==3.2.1
joblib==1.0.1
fasttext==0.9.2
pandas==1.3.4
numpy==1.19.5
scipy==1.7.1
scikit-learn==0.23.2

In [ ]:
label, probability

In [ ]:
pip list |grep scipy

In [ ]:
df = pd.DataFrame.from_dict(
    {'product_name': {0: 'Парацетамол таблетки 500 мг 10 шт., упаковки ячейковые контурные (2), пачки картонные, рег № ЛС-001364 от 06.08.2010, серия 190618, партия 59110 упаковок, годен до 01.07.2022, производства  ОАО "Фармстандарт-Лексредства", ИНН 4631002737, 305022, Курская область, Курск, ул. 2-я Агрегатная, 1А/18, Россия, код ОКПД2 21.20.10.232 ',
      1: 'Перезаряжаемая литий-ионная батарея торговой марки HUAWEI модель HB642735ECW',
      2: 'Перезаряжаемая литий-ионная батарея торговой марки vivo модель B-E8',
      3: 'Аппарат вакуумно-лазерной терапии стоматологический АВЛТ-"ДЕСНА" (по методу Кулаженко-Лепилина)',
      4: 'Блоки оконные и балконные дверные из алюминиевых профилей системы "INICIAL" серии "IW63" фирмы ООО "Урало-Сибирская профильная компания"'},
     'category_1': {0: 9300, 1: 3482, 2: 3482, 3: 9444, 4: 5270}
    })

product_name_clf = ProductNameClassifier()


#Скоринг всего файла без метки от пользователя
df[['model_label', 'probability']] = None
df[['model_label', 'probability']] = df[['product_name']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")
print('3:\n', df.head())
#Скоринг всего файла с наличием метки от пользователя
df[['model_label', 'probability', 'is_equal']] = None
df[['model_label', 'probability', 'is_equal']] = df[['product_name', 'category_1']].apply(lambda x: product_name_clf.predict(*x),
                                                                                                    axis=1,
                                                                                                    result_type="expand")
print('3:\n', df.head())